# Classification on `emnist`

## 1. Create `Readme.md` to document your work

Explain your choices, process, and outcomes.

## 2. Classify ~~all symbols~~ letters a -> g

### Subset the data

Select only the lowercase letters (a, b, ..., g) for classification

### Choose a model

Your choice of model! Choose wisely...

### Train away!

Is do you need to tune any parameters? Is the model expecting data in a different format?

### Evaluate the model

Evaluate the models on the test set, analyze the confusion matrix to see where the model performs well and where it struggles.

### Investigate subsets

On which classes does the model perform well? Poorly? Evaluate again, excluding easily confused symbols (such as 'O' and '0').

### Improve performance

Brainstorm for improving the performance. This could include trying different architectures, adding more layers, changing the loss function, or using data augmentation techniques.

## 3. Model showdown: upper vs lowercase on abcXYZ

### Subset the data

Select out the set of upper- and lowercase (a, b, c, x, y z, A, B, C, X, Y, Z). Note that some of these classes can be confusing (e.g., x and y).

### Train and tune models

Perform a full model training and hyperparameter tuning.

1. Select candidate models, hyperparameter options, and evaluation metric
2. Set aside a validation hold-out dataset
3. Train models over K splits (use k-fold or train/test split)
    1. Split train using k-fold with the number of folds equal to the number of parameter combinations
    2. Train on k-fold split
    3. Record performance of each set of parameters
    4. Use winning set of parameters to train model on full training set
    5. Record each model's performance on that split's test set
4. Evaluate model performance and promote one model as the winner
5. Train winning model on both train + test
6. Check model performance on the validation hold-out


## 4. (_Optional_) Model comparison: classify even vs odd

**NOTE:** This is a larger dataset (~400k rows) so it will require more memory and time to train models on it. 

Alternatively, you can train models on smaller subsets of the data to get a feel for which models perform better than others. Then train the winning model on the full dataset and validate against the hold-out.

### Subset the data

Select only digits and add a column for 'is_even'. Be sure to create a validation hold-out dataset for later.

### Build and compare models

Train at least two different models, compare the results and choose a winner based on an evaluation metric of your choice.

In [99]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: emnist in c:\users\wilso\onedrive\documents from long ago\datasci 223\datasci_223\.venv\lib\site-packages (from -r requirements.txt (line 1)) (0.0)
  Using cached pyarrow-15.0.0-cp312-cp312-win_amd64.whl.metadata (3.1 kB)



ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [100]:
# Import packages
import os
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import emnist
from IPython.display import display, Markdown

# ML packages
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, ParameterGrid, train_test_split

# XGBoost (SVM)
from xgboost import XGBClassifier
# Deep Learning
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Flatten
# Constants
SIZE = 28

In [101]:
# Define helper functions
def int_to_char(label):
    """Convert an integer label to the corresponding uppercase character."""
    if label < 10:
        return str(label)
    elif label < 36:
        return chr(label - 10 + ord('A'))
    else:
        return chr(label - 36 + ord('a'))

def which_case(case):
    """Group lower cases as 0, and upper cases as 1"""
    if np.char.islower(case):
        return 0
    else:
        return 1
    
def show_image(row):
    """Display a single image and its corresponding label."""
    image = row['image']
    label = row['label']
    plt.imshow(image, cmap='gray')
    plt.title('Label: ' + int_to_char(label))
    plt.axis('off')
    plt.show()

def show_grid(data, title=None, num_cols=5, figsize=(20, 10)):
    """
    Display a list of images as a grid of num_cols columns.
    images: a list of images, each represented as a 28x28 numpy array
    labels: a list of labels, one for each image
    title: (optional) a title for the plot
    num_cols: (optional) number of columns to use in the grid
    figsize: (optional) size of the figure
    """
    num_images = len(data)
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    if title is not None:
        fig.suptitle(title, fontsize=16)
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            if index < num_images:
                axes[i, j].imshow(data.iloc[index]['image'], cmap='gray')
                axes[i, j].axis('off')
                label = int_to_char(data.iloc[index]['label'])
                axes[i, j].set_title(label)
    plt.show()

# Get a random image of a given label from the dataset
def get_image_by_label(data, label):
    """Get a random image of a given label from the dataset."""
    images = data[data['label'] == label]['image'].tolist()
    return random.choice(images)

# Plot the training and validation accuracy during the training of a model
def plot_accuracy(history):
    """Plot the training and validation accuracy during the training of a model."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot the training and validation loss during the training of a model
def plot_loss(history):
    """Plot the training and validation loss during the training of a model."""
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Normalize the pixel values of the images in the dataset to have zero mean and unit variance
# This is a common preprocessing step for neural networks, but may not be necessary in all cases
def normalize_images(images):
    """Normalize the pixel values of the images in the dataset to have zero mean and unit variance."""
    images = np.array(images)
    mean = images.mean()
    std = images.std()
    images = (images - mean) / std
    return images.tolist()

# Display metrics for a model
def display_metrics(task, model_name, metrics_dict):
    """Display performance metrics and confusion matrix for a model."""
    metrics_df = pd.DataFrame()
    cm_df = pd.DataFrame()
    for key, value in metrics_dict[task][model_name].items():
        if type(value) == np.ndarray:
            cm_df = pd.DataFrame(value, index=['actual 0', 'actual 1'], columns=['predicted 0', 'predicted 1'])
        else:
            metrics_df[key] = [value]
    display(Markdown(f'# Performance Metrics: {model_name}'))
    display(metrics_df)
    display(Markdown(f'# Confusion Matrix: {model_name}'))
    display(cm_df)

In [102]:
# extract both training and test from emnist
image, label = emnist.extract_training_samples('byclass')
raw_train = pd.DataFrame()
raw_train['label'] = label
raw_train['image'] = list(image)
image, label = emnist.extract_test_samples('byclass')
raw_test = pd.DataFrame()
raw_test['label'] = label
raw_test['image'] = list(image)
print(raw_train.shape)
raw_test.shape
# combine training and test
raw_data = pd.concat([raw_train,raw_test])
print(raw_data.shape)
#create a 10% subset of raw_data
data = raw_data.sample(frac=0.1, replace=False, random_state=1)
data.shape
# add column "class" 
class_label = np.array([int_to_char(l) for l in data.iloc[:,0]])
# make a copy of data added with class and image_flat cols
data2 = data
data2['class'] = class_label
# add image_flat
data2['image_flat'] = data2['image'].apply(lambda x: np.array(x).reshape(-1))
# Split to train:test = 7:3
train = data2.iloc[0:56998, :]
test = data2.iloc[56998:, :]

(697932, 2)
(814255, 2)


In [103]:
# Initialize random forest classifier
rf_clf = RandomForestClassifier(n_estimators=20, random_state=5, max_depth=20)
# Train and evaluate model
rf_clf.fit(train['image_flat'].tolist(), train['class'])
y_pred = rf_clf.predict(test['image_flat'].tolist())
# Calculate performance metrics
acc = accuracy_score(test['class'], y_pred)
prec = precision_score(test['class'], y_pred, average='macro')
rec = recall_score(test['class'], y_pred, average='macro')
f1 = f1_score(test['class'], y_pred, average='macro')
cm = confusion_matrix(test['class'], y_pred)
print([acc,prec,rec,f1])

c:\Users\wilso\OneDrive\Documents from long ago\Datasci 223\datasci_223\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[0.7480759783854594, 0.6389624182436264, 0.5219646237835093, 0.5433253811985825]


In [104]:
# Subset for upper- and lowercase letters a, b, c, x, y, z
abcxyz = data2[(data2['class'].isin(['a', 'b', 'c', 'A', 'B', 'C', \
                                               'x', 'y', 'z', 'X', 'Y', 'Z']))]
# add column "case" 
case_label = np.array([which_case(l) for l in abcxyz.iloc[:,2]])
abcxyz['case'] = case_label

C:\Users\wilso\AppData\Local\Temp\ipykernel_24200\2772864361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abcxyz['case'] = case_label


In [105]:
# Data splitting
X=abcxyz['image_flat'].tolist()
y=abcxyz['case'].tolist()
# Select candidate models, hyperparameter options, and evaluation metric
rf_param_grid = {'n_estimators': [10, 20], 'max_depth': [10, 20], 'random_state': [5, 5]}
xgb_param_grid = {'n_estimators': [10, 20], 'max_depth': [3, 5], 'learning_rate': [0.3, 0.4], 'random_state': [5, 5]}

scoring_metric = 'accuracy'
# Split data into train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Define how many folds to split on
kf = KFold(n_splits=2, shuffle=True, random_state=42)
models = [
    (RandomForestClassifier(), rf_param_grid),
    (XGBClassifier(), xgb_param_grid)
]
# Initialize best score and best model
best_score = 0
best_model = None

In [106]:
for model, param_grid in models:
    for param_combination in ParameterGrid(param_grid):
        model.set_params(**param_combination)
 # cross_val_score returns array of scores, which you need to take the avg
        scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_model = model.get_params()
# Print the best model
print("Best Model:", best_model)
best_score

Best Model: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 20, 'n_jobs': None, 'oob_score': False, 'random_state': 5, 'verbose': 0, 'warm_start': False}


0.7984024343856979

In [107]:
randfor_clf = RandomForestClassifier(n_estimators=20, random_state=5, max_depth=10)
randfor_clf.fit(X_train, y_train)
y_val_pred = randfor_clf.predict(X_val)
validation_score = accuracy_score(y_val, y_val_pred)
print(f"Validation Hold-out Score: {validation_score}")

Validation Hold-out Score: 0.823574144486692
